Importieren der benötigten Packages

In [171]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from matplotlib import rcParams

# Schriftart und Layout einstellen
rcParams.update({
    "text.usetex": True,  # Damit LaTeX-Schrift verwendet wird
    "font.family": "sans-serif",
    "font.sans-serif": ["Latin Modern Sans"],
    "font.size": 10,      # Schöne Standardgröße
    "figure.dpi": 300,
    "figure.figsize": (6.0, 4.0),  # ca. 15cm breit -> perfekte Breite für A4-Seite in LaTeX
    "savefig.bbox": "tight"        # Speichert das Bild ohne überflüssigen Rand
})



Laden der Daten

In [172]:

# Laden der Daten aus der CSV-Datei
file_path = '../data/Optimierung_Transformation_Basiskoordinatensystem.csv'
file_path_G2 = '../data/Messpunkte_G2_Optimierung_Transformation.csv'
data = pd.read_csv(file_path, delimiter=';', decimal='.')
data_G2 = pd.read_csv(file_path_G2, delimiter=';', decimal='.')

# Auswahl der relevanten Spalten für die Transformation
encoder_points = data[['x_Encoder', 'y_Encoder', 'z_Encoder']].values
ts_points = data[['x_TS', 'y_TS', 'z_TS']].values

encoder_points_G2 = data_G2[['x_Encoder', 'y_Encoder', 'z_Encoder']].values
delta_points_G2 = data_G2[['delta_x', 'delta_y', 'delta_z']].values


Funktionen für die Optimierung

In [173]:
# Funktionen für die Ermittlung der optimalen Transformation
def transform_points(params, points):
    tx, ty, tz = params[:3]  # Translation
    rx, ry, rz = params[3:]  # Rotationswinkel

    # Rotationsmatrizen bei Verwendung von Euler Winkeln (xyz-Konvention)
    Rx = np.array([[1, 0, 0],
                   [0, np.cos(rx), -np.sin(rx)],
                   [0, np.sin(rx), np.cos(rx)]])
    Ry = np.array([[np.cos(ry), 0, np.sin(ry)],
                   [0, 1, 0],
                   [-np.sin(ry), 0, np.cos(ry)]])
    Rz = np.array([[np.cos(rz), -np.sin(rz), 0],
                   [np.sin(rz), np.cos(rz), 0],
                   [0, 0, 1]])
    R = Rz @ Ry @ Rx  # Combined rotation matrix

    # Apply rotation and translation
    transformed_points = points @ R.T + np.array([tx, ty, tz])
    return transformed_points

# Define the cost function to minimize the deltas
def cost_function(params):
    transformed_points = transform_points(params, encoder_points)
    error = ts_points - transformed_points
    return np.sum(np.linalg.norm(error, axis=1))


Optimierung zur Ermittlung der besten Parameter für die Transformation des Basiskoordinatensystems

In [174]:
# Initialwerte für die Optimierung [tx, ty, tz, rx, ry, rz]
initial_guess = ([ 0,0,0,0,0,0])

# Optimierung der Parameter mittels der BFGS-Methode. Kostenfunktion wird durch Änderung der Parameter minimiert.
result = minimize(cost_function, initial_guess, method='BFGS')

# Ergebnisse der Optimierung
optimized_params = result.x
translation = optimized_params[:3]
rotation = optimized_params[3:]

print("Optimierte Translation (tx, ty, tz):", translation)
print("Optimierte Rotation (rx, ry, rz in Radianten):", rotation)

Optimierte Translation (tx, ty, tz): [  0.24238799   0.73412269 -15.48420945]
Optimierte Rotation (rx, ry, rz in Radianten): [0.00048722 0.00078978 0.00585483]


Ermittelte Transformation auf Messpunkte anwenden

In [175]:
# Transformieren der Encoder-Punkte mit den optimierten Parametern
transformed_encoder_points = transform_points(optimized_params, encoder_points)

# Speichern der transformierten Punkte im DataFrame und in einer CSV-Datei
data[['x_Transformed', 'y_Transformed', 'z_Transformed']] = transformed_encoder_points
#data.to_csv('../results/Tabellen/Transformed_Data.csv', index=False)

Berechnung des RMS-Fehlers, des Fehlermittelwertes und der Standardabweichung des Fehlers vor und nach der Transformation

In [176]:
# Fehlerberechnungen
errors_before = ts_points - encoder_points
errors_after = ts_points - transformed_encoder_points

# Normen der Fehlervektoren
norms_before = np.linalg.norm(errors_before, axis=1)
norms_after = np.linalg.norm(errors_after, axis=1)

# RMS Fehler der Normen
rms_error_norm_before = np.sqrt(np.mean(norms_before**2))
rms_error_norm_after = np.sqrt(np.mean(norms_after**2))

# RMS Fehler pro Achse
rms_error_per_axis_before = np.sqrt(np.mean(errors_before**2, axis=0))
rms_error_per_axis_after = np.sqrt(np.mean(errors_after**2, axis=0))

# Mittlerer Fehler pro Achse
mean_error_per_axis_before = np.mean(errors_before, axis=0)
mean_error_per_axis_after = np.mean(errors_after, axis=0)

# Standardabweichung der Fehler pro Achse
std_error_per_axis_before = np.std(errors_before, axis=0)
std_error_per_axis_after = np.std(errors_after, axis=0)

# Maximaler Fehler der Normen
max_norm_before = np.max(np.linalg.norm(errors_before, axis=1))
max_norm_after = np.max(np.linalg.norm(errors_after, axis=1))

# Erstellen eines DataFrames
data_final = {
    "Metric": [
        "RMS Fehler der Norm",
        "RMS Fehler X",
        "RMS Fehler Y",
        "RMS Fehler Z",
        "Mittlerer Fehler X",
        "Mittlerer Fehler Y",
        "Mittlerer Fehler Z",
        "StdAbw Fehler X",
        "StdAbw Fehler Y",
        "StdAbw Fehler Z",
        "Maximaler Fehler der Norm",
    ],
    "Vor Transformation": [
        rms_error_norm_before,
        rms_error_per_axis_before[0],
        rms_error_per_axis_before[1],
        rms_error_per_axis_before[2],
        mean_error_per_axis_before[0],
        mean_error_per_axis_before[1],
        mean_error_per_axis_before[2],
        std_error_per_axis_before[0],
        std_error_per_axis_before[1],
        std_error_per_axis_before[2],
        max_norm_before,
  ],
    "Nach Transformation": [
        rms_error_norm_after,
        rms_error_per_axis_after[0],
        rms_error_per_axis_after[1],
        rms_error_per_axis_after[2],
        mean_error_per_axis_after[0],
        mean_error_per_axis_after[1],
        mean_error_per_axis_after[2],
        std_error_per_axis_after[0],
        std_error_per_axis_after[1],
        std_error_per_axis_after[2],
        max_norm_after,
    ],
}

df = pd.DataFrame(data_final)

# Ausgabe der Tabelle
print(df.to_string(index=False, float_format="%.4f"))

# DataFrame für die Ergebnisse erstellen
df_results = pd.DataFrame({
    'Points': data['Point_Encoder'],
    'Encoder_X': encoder_points[:, 0],
    'Encoder_Y': encoder_points[:, 1],
    'Encoder_Z': encoder_points[:, 2],
    'Error_X_Before': errors_before[:, 0],
    'Error_Y_Before': errors_before[:, 1],
    'Error_Z_Before': errors_before[:, 2],
    'Transformed_Encoder_X': transformed_encoder_points[:, 0],
    'Transformed_Encoder_Y': transformed_encoder_points[:, 1],
    'Transformed_Encoder_Z': transformed_encoder_points[:, 2],
    'Error_X_After': errors_after[:, 0],
    'Error_Y_After': errors_after[:, 1],
    'Error_Z_After': errors_after[:, 2],
    'Norm_Error_Before': norms_before,
    'Norm_Error_After': norms_after
})

# Speichern der Ergebnisse in einer CSV-Datei
df_results.to_csv('../results/Tabellen/Transformation_Fehler_Optimierung_Basiskoordinatensystem.csv', index=False)


                   Metric  Vor Transformation  Nach Transformation
      RMS Fehler der Norm             40.1312               9.2902
             RMS Fehler X             37.4611               4.9951
             RMS Fehler Y              6.4695               3.4481
             RMS Fehler Z             12.8577               7.0333
       Mittlerer Fehler X            -36.5245              -0.2079
       Mittlerer Fehler Y             -1.1202              -0.4717
       Mittlerer Fehler Z            -10.7749               1.4945
          StdAbw Fehler X              8.3245               4.9908
          StdAbw Fehler Y              6.3718               3.4157
          StdAbw Fehler Z              7.0158               6.8727
Maximaler Fehler der Norm             72.6811              28.4076


Erstellen eines Plots, welcher die Achsenfehler vor und nach der Transformation für jede Achse zeigt

In [177]:
# Plot zur Visualisierung der Achsenfehler vor und nach der Transformation
fig, axes = plt.subplots(3, 1, figsize=(12, 15), sharex=True)

# Plot der Fehler für die X-Achse
axes[0].plot(errors_before[:, 0], label='Vor Transformation', color='red', marker='o', linestyle='dashed')
axes[0].plot(errors_after[:, 0], label='Nach Transformation', color='green', marker='x', linestyle='dotted')
axes[0].set_ylabel('X-Fehler in mm')
axes[0].legend()
axes[0].grid(True)

# Plot der Fehler für die Y-Achse
axes[1].plot(errors_before[:, 1], label='Vor Transformation', color='red', marker='o', linestyle='dashed')
axes[1].plot(errors_after[:, 1], label='Nach Transformation', color='green', marker='x', linestyle='dotted')
axes[1].set_ylabel('Y-Fehler in mm')
axes[1].legend()
axes[1].grid(True)

# Plot der Fehler für die Z-Achse
axes[2].plot(errors_before[:, 2], label='Before Transformation', color='red', marker='o', linestyle='dashed')
axes[2].plot(errors_after[:, 2], label='After Transformation', color='green', marker='x', linestyle='dotted')
axes[2].set_ylabel('Z-Fehler in mm')
axes[2].set_xlabel('Punkt Index')
axes[2].legend()
axes[2].grid(True)

plt.tight_layout()
plt.savefig('../results/Diagramme/Achsenfehler_vor_nach_Optimierung.pdf')  # <<< PDF speichern
plt.close()



3D-Vektorplot zur Darstellung der Lage der Punkte und deren Abweichungen vor und nach Optimierung

In [ ]:
# Definieren der Abweichungen in mm
scaled_errors_before = errors_before 
scaled_errors_after = errors_after 
scaled_errors_G2 = delta_points_G2 

sizeref_factor = 10  # Skalierfaktor für die Pfeile

# Werte der Messreihe für den Plot definieren
quivers_before = go.Cone(
    x=encoder_points[:, 0],
    y=encoder_points[:, 1],
    z=encoder_points[:, 2],
    u=scaled_errors_before[:, 0],
    v=scaled_errors_before[:, 1],
    w=scaled_errors_before[:, 2],
    opacity= 0.6,
    sizemode="raw",
    sizeref=sizeref_factor, 
    colorscale='Reds',
    name='Vorher',
    legend= "legend",
    colorbar=dict(
        title="Vorher:<br>Abweichung<br>in mm ",
        len=0.3,  # Länge des Colorbars
        x=0.85,   # X-Position des Colorbars
        y=0.9     # Y-Position des Colorbars
    ),
    )

# Werte der Messreihe für den Plot definieren
quivers_after = go.Cone(
    x=transformed_encoder_points[:, 0],
    y=transformed_encoder_points[:, 1],
    z=transformed_encoder_points[:, 2],
    u=scaled_errors_after[:, 0],
    v=scaled_errors_after[:, 1],
    w=scaled_errors_after[:, 2],
    sizemode="raw",
    opacity= 1,
    sizeref=sizeref_factor, 
    colorscale='Greens',
    name='Nachher',
    legend= "legend2",
    colorbar=dict(
        title="Nachher:<br>Abweichung<br>in mm",
        len=0.3,  # Länge des Colorbars
        x=0.85,  # X-Position des Colorbars
        y=0.6     # Y-Position des Colorbars
    )
    )

# Werte der Messreihe für den Plot definieren
quivers_G2 = go.Cone(
    x=encoder_points_G2[:, 0],
    y=encoder_points_G2[:, 1],
    z=encoder_points_G2[:, 2],
    u=scaled_errors_G2[:, 0],
    v=scaled_errors_G2[:, 1],
    w=scaled_errors_G2[:, 2],
    opacity= 0.3,
    sizemode="raw",
    sizeref=sizeref_factor, 
    colorscale='Blues',
    name='Vorher',
    legend= "legend",
    colorbar=dict(
        title="Abweichung G2<br>in mm ",
        len=0.3,  # Länge des Colorbars
        x=0.85,   # X-Position des Colorbars
        y=0.3    # Y-Position des Colorbars
    ),
    )

# Messreihen dem Plot hinzufügen
fig = go.Figure(data=[quivers_before, quivers_after, quivers_G2])

# Layout anpassen
fig.update_layout(
    title='3D Vektorplot der Abweichungen vor und nach der Transformation',
    scene=dict(
        xaxis_title='<i>x</i> in mm',
        yaxis_title='<i>y</i> in mm',
        zaxis_title='<i>z</i> in mm',
           ),
    font=dict( 
       family="Latin Modern Sans",
    ),
)


# Speichern des Plots in Form eines HTML files
fig.write_html("../results/Diagramme/3D_Vektor_Plot_Optimierung_Basiskoordinatensystem.html")